# ***-1-Importing the recommended libraries***

In [10]:
import numpy as np 
from pandas import DataFrame as df
from binarytree import Node
import scipy
Letters="ACGT-" # required for owr operations

# ***-2-Create the recommended functions***

#

### **->Test sequences function** 

In [11]:
def test(seqs):
    seq_length = len(seqs[0])
    if not all(len(seq) == seq_length for seq in seqs):
        raise ValueError("Input sequences must be have the same length")
    elif len(seqs)<2:
        raise ValueError("The input sequences  must be need to add more  ")


# ***->generate_count_matrix***

In [12]:
def generate_count_matrix(sequences=["AC-GT", "AC-GT", "GC-AT"]):
    # Check that all sequences have the same length
    seq_length = len(sequences[0])
    test(sequences)
    # Initialize count matrix with zeros
    count_matrix = {letter: [0] * seq_length for letter in Letters}
    # Update count matrix based on aligned sequences
    for i in range(seq_length):
        column = [seq[i] for seq in sequences]
        for letter in "ACGT-":
            count_matrix[letter][i] = column.count(letter)/len(column)
    return count_matrix

#   -> ***test if its transition or neither***

In [13]:
def is_transition(n_1, n_2): # n:nucleotide
    purines = {'A', 'G'} # for purines transition
    pyrimidines = {'C', 'T'} #  for pyrimidines transition
    test=lambda a,b,c:(a in c) and (b in c)
    return (test(n_1,n_2,purines)) or (test(n_1,n_2, pyrimidines))

### -> **calculate_score (Note :  the values are from the course , see the Readme file)**

In [14]:
def calculate_score(item, index,count_matrix):
    
    # from the course
    
    MNuc = 2 # une correspondence impliquant deux nucléotides identiques
    
    MGap = 1 #  une correspondance comprenant deux gaps
    
    MMs = 1 #  l'une impliquant une différence de transition
    
    MMv = -1 # l'autre une différence transversale
    
    gap_penalty = -3
    current_score = 0

    for letter, counts in count_matrix.items():
        if letter == item:
            if item == '-':  based_weight = MGap
            else: based_weight = MNuc
            
        elif letter == '-' or item == '-':  based_weight=gap_penalty
        
        elif is_transition(letter,item):  based_weight=MMs
        
        else:  based_weight=MMv
        
        current_score += counts[index] * based_weight

    return current_score


# **->Profile generation**

In [15]:
def Profile(aligned_sequences, seq_T = "ACG"): 
    """_summary_

    Args:
        aligned_sequences (list): aligned_sequences is the list of our sequences
        seq_T (str, optional):Target Sequence. Defaults to "ACG".

    Returns:
        list: sequences_results
    """
    m, n = len(seq_T), len(aligned_sequences[0])
    # init a score matrix
    score_matrix = [[0] * (n + 1) for _ in range(m + 1)] 
    gap_penalty = -3
    # init a direction matrix
    direction_matrix = [[''] * (n + 1) for _ in range(m + 1)]
    mito=np.array(score_matrix)
    show=lambda a:df(a,index=list("-"+seq_T) , columns=["-"]+[f"C{i}" for i in range(len(aligned_sequences[0]))])
    
    print("count_matrix :")
    count_matrix = generate_count_matrix(aligned_sequences)
    # Print the count matrix
    print(df(count_matrix.values(),index=count_matrix.keys() , columns=[f"C{i}" for i in range(len(aligned_sequences[0]))]))
    diro=np.array(direction_matrix)
    # print(df(diro))
        # Initialize the first column of score_matrix and direction_matrix
    for i in range(1, m + 1):
        score_matrix[i][0] = score_matrix[i - 1][0] + gap_penalty
        direction_matrix[i][0] = '↑'

    # Initialize the first row of score_matrix and direction_matrix
    for j in range(1, n + 1):
        score_matrix[0][j] = score_matrix[0][j - 1] + gap_penalty
        direction_matrix[0][j] = '←'

    for i in range(1, m + 1):
        for j in range(1, n + 1):

            delete = score_matrix[i - 1][j] + calculate_score('-', j - 1,count_matrix)
            insert = score_matrix[i][j - 1] + calculate_score('-', j - 1,count_matrix)

            match = score_matrix[i - 1][j - 1] + calculate_score(seq_T[i - 1], j - 1,count_matrix)

            max_score = max(match, delete, insert)

            score_matrix[i][j] = max_score
            if max_score == match:
                direction_matrix[i][j] += '↖'
            if max_score == delete:
                direction_matrix[i][j] += '↑'
            if max_score == insert:
                direction_matrix[i][j] += '←'
    print("Score Matrix:")
    mito=np.array(score_matrix)
    print(show(mito))


    print("\nDirection Matrix:")
    diro=np.array(direction_matrix)
    print(show(diro))

    sequences_results = []

    def backward(i, j, seq_align):
        # global all_alignments

        if i == 0 and j == 0:
            sequences_results.append(seq_align[::-1])
            return

        if '↖' in direction_matrix[i][j]:
            backward(i - 1, j - 1, seq_align + seq_T[i - 1])
        if '↑' in direction_matrix[i][j]:
            backward(i - 1, j, seq_align + '-')
        if '←' in direction_matrix[i][j]:
            backward(i, j - 1, seq_align + '-')

    backward(m, n, '')

    return sequences_results



# ***Example usage***

In [16]:
# Example usage:
aligned_sequences = ["AC-GT", "AC-GT", "GCCAT"]
# print("count_matrix :")
count_matrix = generate_count_matrix(aligned_sequences)
# Print the count matrix
# print(df(count_matrix.values(),index=count_matrix.keys() , columns=[f"C{i}" for i in range(len(aligned_sequences[0]))]))
# aligned_sequences = ["AC-GT", "AG-CT", "GCCAT","AATCG"]
T = "ACG"
aligned_result = Profile(aligned_sequences, )
print("Aligned_result :")
for r in aligned_sequences:
    print(r)
print(aligned_result)

count_matrix :
         C0   C1        C2        C3   C4
A  0.666667  0.0  0.000000  0.333333  0.0
C  0.000000  1.0  0.333333  0.000000  0.0
G  0.333333  0.0  0.000000  0.666667  0.0
T  0.000000  0.0  0.000000  0.000000  1.0
-  0.000000  0.0  0.666667  0.000000  0.0
Score Matrix:
     -        C0        C1        C2         C3         C4
-  0.0 -3.000000 -6.000000 -9.000000 -12.000000 -15.000000
A -3.0  1.666667 -1.333333 -1.666667  -4.666667  -7.666667
C -6.0 -1.333333  3.666667  3.333333   0.333333  -2.666667
G -9.0 -4.333333  0.666667  3.000000   5.000000   2.000000

Direction Matrix:
   - C0 C1 C2 C3 C4
-     ←  ←  ←  ←  ←
A  ↑  ↖  ←  ←  ←  ←
C  ↑  ↑  ↖  ←  ←  ←
G  ↑  ↑  ↑  ↑  ↖  ←
Aligned_result :
AC-GT
AC-GT
GCCAT
['AC-G-']
